In [1]:
import pydantic
from pydantic import BaseModel
import io
import json
import pandas as pd
from copy import deepcopy

from typing import TypeAlias, Any
from collections.abc import Iterable, Sequence
import datetime
from datetime import datetime as Datetime
from datetime import timedelta as Timedelta
from datetime import date as Date
from datetime import timezone
from zoneinfo import ZoneInfo

# time handling module
import time_handling
from time_handling import TIMEZONE_CEST

In [2]:
def clean_prop_dict(
    dictionary: dict[str, Any],
    prop_list: list[str],
) -> dict[str, Any]:
    """removes key-value paris from a dictionary by a provided list
    which contains the keys that are to be deleted

    Parameters
    ----------
    dictionary : dict[str, Any]
        dictionary whose key-value pairs should be deleted
    prop_list : list[str]
        list of keys which should be removed from the given dictionary

    Returns
    -------
    dict[str, Any]
        cleaned dictionary
    """
    dictionary = deepcopy(dictionary)
    prop_list = prop_list.copy()
    
    for prop in prop_list:
        
        if prop in dictionary:
            del dictionary[prop]
    
    return dictionary

## General idea
- define interface to parse given data also in not desired formats
- try parsing primarily text-based sources (JSON-like), not already structured and parsed as Data Frame or similar format
- provide common interface which does not heavily depend on several preprocessing steps

#### Update 2023-12-11
- Pydantic parsing not sufficient
- need to build general adapter class with common parsing functionality
- build special adapter class which allows for more specific transformation (for each data source separately), inherits from base adapter class
- finally provide parsed information for pydantic data container

work flow as follows:
``Data Set`` --> ``Specific Adapter Class`` --> ``Pydantic Model`` (data container)

- time zone handling:
    - calculation must be done in UTC
    - display to user as datetime with converted time zone

In [7]:
dt = Datetime(2023, 10, 28, 23, tzinfo=TIMEZONE_CEST)
#dt = time_handling.current_time_tz(tz=TIMEZONE_CEST)
#dt = Datetime(2023, 10, 28, 23)
print(dt)

2023-10-28 23:00:00+02:00


In [8]:
td = Timedelta(hours=10)

In [9]:
dt2 = time_handling.add_timedelta_with_tz(
    starting_dt=dt,
    td=td,
)

In [10]:
print(dt2)

2023-10-29 08:00:00+01:00


In [179]:
# transform to utc
dt_utc = dt.astimezone(TIMEZONE_UTC)

# add duration
dt2 = dt_utc + td

# transform back
dt2 = dt2.astimezone(TIMEZONE_CEST)

In [180]:
print(dt_utc)
print(dt2)

2023-10-28 21:00:00+00:00
2023-10-29 08:00:00+01:00


In [168]:
td = Timedelta(hours=10)

In [169]:
print(dt + td)

2023-10-29 09:00:00+01:00


In [109]:
CustomID: TypeAlias = int | str

Conclusions:
- use more precise naming scheme
- attributes do not have to correspond to the naming scheme of the simulation environment
- Pydantic model defines entry point to simulation environment (gold standard)
    - model can easily be fitted in the job class
- therefore: data transformation and mapping to model attributes necessary
    - example: ``Bearbeitungszeit in Stunden`` --> ``processing time``

In [130]:
class Job(BaseModel):
    exec_systems_order: Sequence[CustomID]
    proc_times: Sequence[Timedelta]
    target_stations_order: Sequence[CustomID | None] | None = None
    setup_times: Sequence[Timedelta | None] | None = None
    prio: int | Sequence[int] | None = None
    planned_starting_date: Datetime | Sequence[Datetime] | None = None
    planned_ending_date: Datetime | Sequence[Datetime] | None = None
    custom_identifier: CustomID | None = None
    name: str | None = None
    state: str = 'INIT'
    additional_info: dict[str, CustomID] | None = None

In [131]:
Job.model_json_schema()

{'properties': {'exec_systems_order': {'items': {'anyOf': [{'type': 'integer'},
     {'type': 'string'}]},
   'title': 'Exec Systems Order',
   'type': 'array'},
  'proc_times': {'items': {'format': 'duration', 'type': 'string'},
   'title': 'Proc Times',
   'type': 'array'},
  'target_stations_order': {'anyOf': [{'items': {'anyOf': [{'type': 'integer'},
       {'type': 'string'},
       {'type': 'null'}]},
     'type': 'array'},
    {'type': 'null'}],
   'default': None,
   'title': 'Target Stations Order'},
  'setup_times': {'anyOf': [{'items': {'anyOf': [{'format': 'duration',
        'type': 'string'},
       {'type': 'null'}]},
     'type': 'array'},
    {'type': 'null'}],
   'default': None,
   'title': 'Setup Times'},
  'prio': {'anyOf': [{'type': 'integer'},
    {'items': {'type': 'integer'}, 'type': 'array'},
    {'type': 'null'}],
   'default': None,
   'title': 'Prio'},
  'planned_starting_date': {'anyOf': [{'format': 'date-time',
     'type': 'string'},
    {'items': {'form

# ***Old***

try to simulate JSON input (text-based)

In [132]:
columns_interest_old = [
    'ID',
    'Artikelnr.',
    'Kap.-gr.',
    'FAKZ',
    'Ag Paketierung',
    'geplanter Termin',
    'Bruttomenge 	in LP',
    'Rüstzeit Logistisch 	in Stunden',
    'Bearbeitungszeit 	in Stunden',
    'Bohrpaketierung',
    'Anzahl Bohr./LP',
    'Fräspaketierung',
    'Anmeldung',
    'Rückmeldung',
]

columns_interest = [
    'ID',
    'Artikelnr.',
    'Kunde',
    'Benennung Kunde',
    'Kapagruppe',
    'Benennung Kapagruppe',
    'FAKZ',
    'AG Paketierung',
    'geplanter Termin (Ende)',
    'Bruttomenge in LP',
    'Rüstzeit',
    'Bearbeitungszeit',
    'AnlageGeplant', # associated machine group, not assigned in this data set
    'Bohrpaketierung',
    'Anzahl Bohrungen je LP',
    'Fräspaketierung',
]

# not reliable: sometimes start and end identical
columns_actual_data = [
    'Anmeldung',
    'Rückmeldung',
]

In [133]:
# import data set
FILE_NAME_DS = './datasets/fcon__2212____01-01-22_17-08-22_renamed.csv'
data = pd.read_csv(FILE_NAME_DS, sep=';', encoding='ansi', 
                   usecols=columns_interest, decimal=',')


In [134]:
data.columns

Index(['ID', 'Artikelnr.', 'Kunde', 'Benennung Kunde', 'Kapagruppe',
       'Benennung Kapagruppe', 'FAKZ', 'AG Paketierung',
       'geplanter Termin (Ende)', 'Bruttomenge in LP', 'Rüstzeit',
       'Bearbeitungszeit', 'AnlageGeplant', 'Bohrpaketierung',
       'Anzahl Bohrungen je LP', 'Fräspaketierung'],
      dtype='object')

#### Simulate job retrieval via JSON

In [135]:
data_extr = data.iloc[:1,:]
# string buffer
buff = io.StringIO()
data_extr.to_json(buff, date_unit='s', orient='records')
string = buff.getvalue()
buff.close()

In [136]:
# load string via Python standard library
json_ret = json.loads(string)
json_ret

[{'ID': 2046707,
  'Artikelnr.': '4-201276p',
  'Kunde': 13158.0,
  'Benennung Kunde': 'KSG GmbH',
  'Kapagruppe': 22162,
  'Benennung Kapagruppe': 'NC-Bohrerei',
  'FAKZ': 'PR',
  'AG Paketierung': 2.0,
  'geplanter Termin (Ende)': '07.12.2020',
  'Bruttomenge in LP': 120,
  'Rüstzeit': 0.17,
  'Bearbeitungszeit': 0.06,
  'AnlageGeplant': None,
  'Bohrpaketierung': '2',
  'Anzahl Bohrungen je LP': 4,
  'Fräspaketierung': '2'}]

In [137]:
job_data = json_ret[0]

In [138]:
# properties which are used directly and 
# therefore should not be included in the 
# job's additional information
props_direct_mapping: list[str] = [
    'Kapagruppe',
    'Bearbeitungszeit',
    'AnlageGeplant',
    'Rüstzeit',
    'geplanter Termin (Ende)',
    'ID',
]

additional_info = clean_prop_dict(dictionary=job_data,
                                  prop_list=props_direct_mapping)
additional_info

{'Artikelnr.': '4-201276p',
 'Kunde': 13158.0,
 'Benennung Kunde': 'KSG GmbH',
 'Benennung Kapagruppe': 'NC-Bohrerei',
 'FAKZ': 'PR',
 'AG Paketierung': 2.0,
 'Bruttomenge in LP': 120,
 'Bohrpaketierung': '2',
 'Anzahl Bohrungen je LP': 4,
 'Fräspaketierung': '2'}

In [140]:
# mapping to interface class
job_repr = Job(
    exec_systems_order = [job_data['Kapagruppe']],
    proc_times = [job_data['Bearbeitungszeit']],
    target_stations_order = [job_data['AnlageGeplant']],
    setup_times = [job_data['Rüstzeit']],
    planned_ending_date = job_data['geplanter Termin (Ende)'],
    custom_identifier = job_data['ID'],
    additional_info = additional_info,
)

ValidationError: 2 validation errors for Job
planned_ending_date.datetime
  Input should be a valid datetime, invalid character in year [type=datetime_parsing, input_value='07.12.2020', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/datetime_parsing
planned_ending_date.json-or-python[json=list[datetime],python=chain[is-instance[Sequence],function-wrap[sequence_validator()]]]
  'str' instances are not allowed as a Sequence value [type=sequence_str, input_value='07.12.2020', input_type=str]

In [84]:
test = Job(**job_0)

ValidationError: 2 validation errors for Job
exec_systems_order
  Field required [type=missing, input_value={'ID': 2046707, 'Artikeln...'Fräspaketierung': '2'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing
proc_times
  Field required [type=missing, input_value={'ID': 2046707, 'Artikeln...'Fräspaketierung': '2'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing

---

### Test "Paketierung"

In [33]:
data = pd.read_csv('./datasets/fcon__2212____01-01-22_17-08-22.csv', 
                   sep=';', encoding='ansi', decimal=',')

C:\Users\flfo\AppData\Local\Temp\ipykernel_5500\3805392045.py:1: DtypeWarning: Columns (67,74,75) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('./datasets/fcon__2212____01-01-22_17-08-22.csv',


In [34]:
data.columns

Index(['FA', 'ID', 'AgNr', 'Artikelnr.', 'Kunde', 'Benennung Kunde', 'Ktr',
       'StAG', 'Benennung AG', 'Kap.-gr.', 'Benennung Kap.-gr.', 'Kst.',
       'Benennung Kst.', 'Ag Kommentar (20.4)', 'Ag Langtext (20.5)',
       'Ag Zusatztext 1 (20.4)', 'Ag Zusatztext 2 (20.4)', 'FAKZ', 'ZU/LosAg',
       'AG-rückgemeldete ZU/LosAg', 'Ag Paketierung', 'geplanter Termin',
       'Anmeldung', 'Rückmeldung', 'Rückmeldung Datum', 'Rückmeldung Uhrzeit',
       'Rückmeldung Tag', 'Rückmeldung Monat', 'Rückmeldung Jahr',
       'Anlieferung', 'Bruttomenge \tin LP', 'LP/ZU', 'LP/LN',
       'Nettofläche/LP \tin dm²', 'Bruttofläche/ZU \tin dm²',
       'Zu-Länge \tin mm', 'Zu-Breite \tin mm', 'Zu-Abmessung', 'KupferLagen',
       'KupferLagenKunde', 'Rüstzeit Logistisch \tin Stunden',
       'Rüstzeit Kalkulatorisch \tin Stunden', 'Bearbeitungszeit \tin Stunden',
       'Logistische Zeit (Bo./Fr.) \tin Stunden', 'AnlageGeplant',
       'AG-rückgemeldete Anlage', 'AL-Cu-Folie (Max) \tin µm',
     

In [47]:
data['Ag Paketierung'] = pd.to_numeric(data['Ag Paketierung'], errors='coerce')
data['Bohrpaketierung'] = pd.to_numeric(data['Bohrpaketierung'], errors='coerce')
data['Fräspaketierung'] = pd.to_numeric(data['Fräspaketierung'], errors='coerce')

In [48]:
subset = ['Ag Paketierung', 'Bohrpaketierung', 'Fräspaketierung']
data = data.dropna(how='any', subset=subset, ignore_index=True)

In [49]:
dtype_map: dict = {
    'Ag Paketierung': int,
    'Bohrpaketierung': int,
    'Fräspaketierung': int,
}

data = data.astype(dtype_map)

In [50]:
data['paketierung_test'] = data['Bohrpaketierung'] - data['Fräspaketierung']

In [58]:
ret1 = (data['Bohrpaketierung'] == data['Ag Paketierung']).sum() / len(data)
ret2 = (data['Fräspaketierung'] == data['Ag Paketierung']).sum() / len(data)

In [62]:
print('Drill:', ret1)
print('Mill:', ret2)

Drill: 0.9997300653241915
Mill: 0.9988662743616045


**Result: packaging attribute almost always the same between drilling or milling and value for whole order**